# Problem Description
**Task**<br>
Find all occurrences of a given collection of patterns in a string.
<br><br>**Input Format**<br>
The first line contains a string Text. The second line specifies an integer 𝑛. The last line
gives a collection of 𝑛 strings Patterns = {𝑝1 , . . . , 𝑝𝑛 } separated by spaces.
<br><br>**Constraints**<br>
All strings contain symbols A, C, G, T only ; 1 ≤ |Text| ≤ 105 ; 1 ≤ 𝑛 ≤ 104 ; ∑︀𝑛𝑖=1 |𝑝𝑖| ≤ 105.
<br><br>**Output Format**<br>
All starting positions (in any order) in Text where a pattern appears as a substring (using 0-based indexing as usual). If several patterns occur at the same position of the Text, still output this position only once.

In [ ]:

global glotext
def find_occurrences(patterns):
    occs = set()
    global resultfinal, glotext

    result = []
    resultList = []
    glotext = glotext+'$'
    suffixArray = build_suffix_array()

    glotext = glotext[:-1]
    for pattern in patterns:
        startEnd = PatternMatchingWithSuffixArray(pattern, suffixArray)
        if startEnd != None:
            resultList.append(startEnd)

    return resultfinal


def build_suffix_array():

  result = []
  global glotext

  order = SortCharacters()
  clas = ComputeCharClasses(order)
  L = 1

  while L<len(glotext):
    order = SortDoubled(L, order, clas)
    clas = UpdateClasses(order, clas, L)
    L = 2*L

  return order

def SortCharacters():
  global glotext
  order = [0]*len(glotext)
  Alpha = sorted(list(set(glotext)))
  count = [0]*len(Alpha)

  for i in range(len(glotext)):
    id = Alpha.index(glotext[i])
    count[id] = count[id] + 1


  for j in range(1, len(Alpha)):

    count[j] = count[j] + count[j-1]
  for i in reversed(range(0, len(glotext))):
    c = Alpha.index(glotext[i])
    count[c] = count[c] -1

    order[count[c]] = i
  return order

def ComputeCharClasses(order):
  global glotext

  clas = [0] * len(glotext)
  for i in range(1, len(glotext)):
    if glotext[order[i]] != glotext[order[i-1]]:
      clas[order[i]] = clas[order[i-1]] +1
    else:
      clas[order[i]] = clas[order[i-1]]
  return clas

def SortDoubled(L, order, clas):
  global glotext
  count = [0] * len(glotext)
  newOrder = [0] * len(glotext)
  for i in range(len(glotext)):
    count[clas[i]] = count[clas[i]] +1
  for j in range(1, len(glotext)):
    count[j] = count[j] + count[j-1]
  for i in reversed(range(0, len(glotext))):
    start = (order[i]-L+len(glotext))%len(glotext)
    cl = clas[start]
    count[cl] = count[cl] -1
    newOrder[count[cl]] = start
  return newOrder

def UpdateClasses(newOrder, clas, L):
  n = len(newOrder)
  newClass = [0]*n
  newClass[newOrder[0]] = 0
  for i in range(1, n):
    cur = newOrder[i]
    prev = newOrder[i-1]
    mid = cur+L
    midPrev = (prev+L)%n
    if clas[cur] != clas[prev] or clas[mid] != clas[midPrev]:
      newClass[cur] = newClass[prev] + 1
    else:
      newClass[cur] = newClass[prev]
  return newClass


'''BWMatching'''

def PreprocessBWT(bwt, pattern):

  sortedbwt = sorted(bwt)
  starts = {}
  occ_count_before = {}
  keys = []
  for i in range(len(bwt)):
    if i != 0:
      for key in occ_count_before.keys():
        occ_count_before[key][i+1] = occ_count_before[key][i]

    if sortedbwt[i] not in keys:
      c = sortedbwt[i]
      starts[c] = i
      keys.append(c)

    if bwt[i] not in occ_count_before.keys():
      c = bwt[i]
      occ_count_before[c] = [0]*(len(bwt)+1)
      occ_count_before[c][i+1] = 1
    else:
      c = bwt[i]
      occ_count_before[c][i+1] += 1

  top = 0
  bottom = len(bwt) - 1

  while top <= bottom:

    if len(pattern) != 0:
      symbol = pattern[-1]
      pattern = pattern[:-1]
      
      if symbol in bwt[top:bottom+1]:
        top = starts[symbol] + occ_count_before[symbol][top]
        bottom = starts[symbol] + occ_count_before[symbol][bottom+1] -1

      else:
        return -1, -1
    else:
      return bottom, top



def BWT(text):
    Matrix = []

    for i in range(len(text)):

        last = text[len(text)-1]
        body = text[:len(text)-1]
        text = last + body
        Matrix.append(text)

    Matrix = sorted(Matrix)
    result = ''
    for i in range(len(Matrix)):
        result += Matrix[i][-1]

    return result

resultfinal = []

def PatternMatchingWithSuffixArray(pattern, suffixArray):
    global glotext
    suffixArray=suffixArray[1:]
    minIndex = 0
    maxIndex = len(glotext)
    while minIndex < maxIndex:
        midIndex = int((minIndex + maxIndex)/2)

        if pattern > glotext[suffixArray[midIndex]:]:

            minIndex = midIndex + 1
        else:
            maxIndex = midIndex

    start = minIndex

    maxIndex = len(glotext)

    while maxIndex > minIndex:
        midIndex = int((minIndex + maxIndex)/2)
        textSuffix = glotext[suffixArray[midIndex]:(suffixArray[midIndex]+len(pattern))]

        if pattern == textSuffix:

            maxIndex = midIndex
            break
        elif pattern < textSuffix:
            maxIndex = midIndex
        else:
            minIndex = midIndex

    if maxIndex >= len(glotext):
        return None

    rref = glotext[suffixArray[maxIndex]+len(pattern)-1]
    
    while pattern[-1] == rref:
        maxIndex += 1
        if maxIndex > len(suffixArray)-len(pattern):
            break
        if suffixArray[maxIndex]+len(pattern)-1 < len(glotext):
            rref = glotext[suffixArray[maxIndex]+len(pattern)-1]
        else:
            break

    maxIndex -= 1
    textSuffix = glotext[suffixArray[maxIndex]:(suffixArray[maxIndex]+len(pattern))]

    if pattern < textSuffix:
        while textSuffix > pattern:
            maxIndex -= 1
            textSufList = []
            for i in range(suffixArray[maxIndex],(suffixArray[maxIndex]+len(pattern)) ):
                if i < len(glotext):
                    textSufList.append(glotext[i])
            textSuffix = ''.join(textSufList)


        maxIndex += 1
    else:

        while textSuffix == pattern:
            maxIndex += 1
            if maxIndex < len(suffixArray):
                textSufList = []
                for i in range(suffixArray[maxIndex],(suffixArray[maxIndex]+len(pattern)) ):
                    if i < len(glotext):
                        textSufList.append(glotext[i])
                textSuffix = ''.join(textSufList)


            else:
                break
    end = maxIndex


    global resultfinal
    if start < end:
        for n in range(start, end):
            if suffixArray[n] not in resultfinal:
                resultfinal.append(suffixArray[n])
glotext = ''

if __name__ == '__main__':
    glotext = input()
    pattern_count = int(input())
    patterns = input().split()

    occs = find_occurrences(patterns)
    print(" ".join(map(str, occs)))
